##### Вопросы для повторения:

* что такое процессы и потоки?
* сколько стеков в многопоточном приложении?
* что такое переключение контекста для ядра?
* когда начинает и заканчивает своё выполнение поток тут:

```c++
int main()
{
    std::thread t([](){ std::cout << "hello from background thread\n"; });
    
    std::cout << "hello from main thread\n";
    
    return 0;
}
```

* какая ошибка в примере выше и как её починить? сколькими вариантами?

    <details>
    <summary>ответ</summary>
    <p>join/detach</p>
    </details>

* что такое эффективность параллелизации? как её вычислить? какое число ожидать?
* что делает и зачем нужен `std::this_thread::yield`?
* зачем нужны `std::promise` - `std::future` и как ими пользоваться?
* что такое `std::async` и как им пользоваться?

##### Замечание с прошлой лекции:

Подробнее про superlinear speedup:<br />
https://en.wikipedia.org/wiki/Speedup#Super-linear_speedup

<br />

### Многопоточность. Часть 2

<br />

##### race condition

https://en.wikipedia.org/wiki/Race_condition

__Вопросы__:
* что такое race condition?
* как с ним бороться?
* каковы гарантии стандарта языка С++ при возникновении race contidition?

__Пример__:

```c++
#include <cassert>
#include <thread>
#include <vector>

int parallel_sum(const std::vector<int>& v, const unsigned threads_count)
{
    const unsigned len = v.size() / threads_count;
    assert(len * threads_count == v.size());

    int rv = 0;

    std::vector<std::thread> threads;
    for (unsigned i = 0; i < threads_count; ++i)
        threads.emplace_back([&](){
            const unsigned start_ix = len * i;
            const unsigned final_ix = len * (i + 1);
            for (unsigned ix = start_ix; ix < final_ix; ++ix)
                 rv += v[ix];
        });

    for (auto& t: threads)
        t.join();

    return rv;
}

int main()
{
    const std::vector<int> v(3'000'000, 1);
    std::printf("sum 1 thr: %d\n", parallel_sum(v, 1));
    std::printf("sum 3 thr: %d\n", parallel_sum(v, 3));
    return 0;
}
```

Возможный вывод:
    
```sh
sum 1 thr: 3000000
sum 3 thr: 1054551
```

Как получается race condition:

```c++
thread_1:             | thread_2:
    read  rv          |     read  rv
    calc  rv + v[i1]  |     calc  rv + v[i2]
    write rv          |     write rv
```

__Замечание__: в примере ещё одна ошибка тут. В чём она?

```c++
std::vector<std::thread> threads;
for (unsigned i = 0; i < threads_count; ++i)
    threads.emplace_back([&](){
        const unsigned start_ix = len * i;
        const unsigned final_ix = len * (i + 1);
        for (unsigned ix = start_ix; ix < final_ix; ++ix)
            rv += v[ix];
    });
```

<details>
<summary>ответ</summary>
<p>
    
Правильный вариант:

```c++
std::vector<std::thread> threads;
for (unsigned i = 0; i < threads_count; ++i)
    threads.emplace_back([len, i, &rv, &v](){
        const unsigned start_ix = len * i;
        const unsigned final_ix = len * (i + 1);
        for (unsigned ix = start_ix; ix < final_ix; ++ix)
            rv += v[ix];
    });
``` 

</p>
</details>

<br />

##### mutex

MUTual EXclusive access primitive

https://en.cppreference.com/w/cpp/thread/mutex

Методы:
    
* `void lock()` - дождаться, пока mutex будет освобождён, и захватить его
* `void unlock()` - освободить захваченный mutex
* `bool try_lock()` - попытка захватить mutex, если он свободен
* `native_handle_type native_handle()` - ОС-специфичный handle (как можно здесь догадаться, mutex - объект ядра ОС)

Вариант исправления задачи с суммой через mutex:

```c++
int parallel_sum(const std::vector<int>& v, const unsigned threads_count)
{
    const unsigned len = v.size() / threads_count;
    assert(len * threads_count == v.size());

    int rv = 0;
    std::mutex mtx;  // synchronization primitive for |rv|

    std::vector<std::thread> threads;
    for (unsigned i = 0; i < threads_count; ++i)
        threads.emplace_back([i, len, &rv, &v, &mtx](){
            const unsigned start_ix = len * i;
            const unsigned final_ix = len * (i + 1);
            for (unsigned ix = start_ix; ix < final_ix; ++ix)
            {
                mtx.lock();    // acquire resource
                rv += v[ix];
                mtx.unlock();  // release resource
            }
        });

    for (auto& t: threads)
        t.join();

    return rv;
}
```


__Замечание__: т.к. переменная `unsigned len` является константой, то в поток её можно передать и по ссылке:
* Константные объекты, имеющиеся до создания фонового потока, можно читать без опасение на race condition
* `unsigned` - маленький объект, его проще передать по копии, чем по ссылке

__Для обсуждения__:
    
1. Никогда не делайте частый доступ до int-переменной через mutex:
    * проблема kernel space
    * более дешёвые альтернативы
    * алгоритм может быть реализован с меньшим числом синхронизаций
2. Какая проблема с парными вызовами `lock()/unlock()`? Подсказка: то же самое что и с `new/delete`?

<br />

##### lock_guard

https://en.cppreference.com/w/cpp/thread/lock_guard

`std::lock_guard` - RAII обёртка над `std::mutex::lock/unlock`.

В конструкторе захватывает mutex, в деструкторе его освобождает.

Второй вариант параллельного суммирования:
    
```c++
int parallel_sum(const std::vector<int>& v, const unsigned threads_count)
{
    const unsigned len = v.size() / threads_count;
    assert(len * threads_count == v.size());

    int rv = 0;
    std::mutex mtx;  // synchronization primitive for |rv|

    std::vector<std::thread> threads;
    for (unsigned i = 0; i < threads_count; ++i)
        threads.emplace_back([i, len, &rv, &v, &mtx](){
            const unsigned start_ix = len * i;
            const unsigned final_ix = len * (i + 1);
            for (unsigned ix = start_ix; ix < final_ix; ++ix)
            {
                std::lock_guard<std::mutex> guard(mtx);  // acquire resource
                rv += v[ix];
            }  // release resource
        });

    for (auto& t: threads)
        t.join();

    return rv;
}
```

<br />

Вариант многопоточного логирования в `std::cout` с предыдущих лекций (чтобы данные между `std::endl` и тексты не перемежались между собой):

```c++
// global mutex for output ostream operations
std::mutex ostream_mutex;

static void print_hello_world()
{
    std::lock_guard<std::mutex> guard(ostream_mutex);
    std::cout << "hello world! (from function)" << std::endl;
}

int main()
{
    std::vector<std::thread> threads;
    for (int i = 0; i < 5; ++i)
        threads.emplace_back(print_hello_world);

    std::lock_guard<std::mutex> guard(ostream_mutex);
    std::cout << "hello world! (from main before threads join)" << std::endl;    

    for (auto& thread : threads)
        thread.join();
    
    std::cout << "hello world! (from main after  threads join)" << std::endl;

    return 0;
}
```

__Вопрос__: где ошибка в коде?

Исправленный вариант:
    
```c++
// global mutex for output ostream operations
std::mutex ostream_mutex;

static void print_hello_world()
{
    std::lock_guard<std::mutex> guard(ostream_mutex);
    std::cout << "hello world! (from function)" << std::endl;
}

int main()
{
    std::vector<std::thread> threads;
    for (int i = 0; i < 5; ++i)
        threads.emplace_back(print_hello_world);

    {
        std::lock_guard<std::mutex> guard(ostream_mutex);
        std::cout << "hello world! (from main before threads join)" << std::endl;
    }

    for (auto& thread : threads)
        thread.join();

    // обратить внимание, что здесь lock_guard не нужен.
    // почему?
    std::cout << "hello world! (from main after  threads join)" << std::endl;

    return 0;
}
```

<br />

##### объекты с доступом из нескольких потоков

Вариант организации многопоточной очереди (есть способы сделать лучше):
    
```c++
template<typename T>
class MTQueue
{
public:
    std::optional<T> pop() {
        std::lock_guard<std::mutex> guard(mtx);
        
        if (queue.empty())
            return std::nullopt;
        
        T x = queue.back();
        queue.pop_back();
        return x;
    }
    
    void push(T x) {
        std::lock_guard<std::mutex> guard(mtx);        
        queue.push_front(std::move(x));        
    }
    
    MTQueue() = default;    
    
    MTQueue(const MTQueue& rhs) {
        std::lock_guard<std::mutex> guard(rhs.mtx);
        queue = rhs.queue;
    }

    MTQueue(MTQueue&& rhs) noexcept {
        std::lock_guard<std::mutex> guard(rhs.mtx);
        queue = std::move(rhs.queue);
    }
    
    ~MTQueue() noexcept {
        // std::lock_guard<std::mutex> guard(mtx);
        queue.clear();
    }

    // мы ещё не готовы реализовать присваивание двух объектов,
    // живущих на разных потоках
    MTQueue& operator = (const MTQueue& rhs) = delete;
    MTQueue& operator = (MTQueue&& rhs) noexcept = delete;
    
private:
    std::deque<T> queue;
    std::mutex mtx;
};
```

__Вопрос__: где в этом коде происходит ужас и кошмар?

<details>
<summary>ответ</summary>
<p>

```c++
~MTQueue() noexcept {
    std::lock_guard<std::mutex> guard(rhs); // <---- тут
    queue.clear();
}
```

Обсудить: если этот `guard` оказался нужен, то что происходит в программе в этот момент?

Пример: перемежающиеся вызовы `push` и `~MTQueue`.
   
</p>
</details>

```c++
MTQueue<int> m


thread_1:           | thread_2:
    m.~MTQueue()    |     m.push(5)
        lock_guard  |         lock_guard
        processing  |         processing
```

__Вопрос__: почему бессмысленно иметь метод `empty`?

Рассмотрим такой вариант:
    
```c++
class MTQueue
{
    ...;
    
    bool empty() {
        std::lock_guard<std::mutex> guard(mtx);

        return queue.empty();
    }
    
    T pop() {
        std::lock_guard<std::mutex> guard(mtx);

        T x = queue.back();
        queue.pop_back();
        return x;
    }
}
```

И вот такой код, исполняющийся параллельно двумя потоками:

```c++
void process_queue(MTQueue& q)   |    void process_queue(MTQueue& q)
{                                |    {
    if (!q.empty())              |        if (!q.empty())
        process_item(q.pop());   |            process_item(q.pop());
}                                |    }
```

Дадим потокам на вход одну и ту же очередь с одним элементов.

<br />

__Упражнение 1__: найдите ошибку в коде. Почему это является ошибкой? Приведите пример.

```c++
class MTSearcher
{
private:
    std::mutex mtx;
    unsigned max_search_result_size;
   
public:
    unsigned get_max_search_result_size() const noexcept
    {
        return max_search_result_size;
    }
    
    ...
};
```

<details>
<summary>подсказка</summary>
<p>
    
```c++
class MTSearcher
{
private:
    std::mutex mtx;
    unsigned max_search_result_size;
   
public:
    unsigned get_max_search_result_size() const noexcept
    {
        return max_search_result_size;
    }
    
    void set_max_search_result_size(unsigned size) noexcept
    {
        std::lock_guard<std::mutex> guard(mtx);
        
        max_search_result_size = size;
    }
    
    ...
};
```

</p>
</details>

<br />

__Упражнение 2__: найдите ошибку в коде


```c++
template<typename T>
class MTQueue
{
private:
    std::mutex mtx;
    std::queue<std::shared_ptr<T>> queue;

public:
    std::shared_ptr<T>& peek() noexcept
    {
        std::lock_guard<std::mutex> guard(mtx);
        return queue.back();
    }
};

// thread 1:
mt_queue.peek() = nullptr;

// thread 2:
std::cout << *mt_queue.peak();
```


Редко когда многопоточный класс может позволить себе такую роскошь как возвращение ссылок/указателей на данные. Как правило, нужно делать defensive copies:

```c++
class MTQueue {
    ...
        
    std::shared_ptr<T> peek() noexcept
    {
        std::lock_guard<std::mutex> guard(mtx);
        return queue.back();
    }
};
```

<br />

__Упражнение 3__: найдите проблему в коде:

```c++
class MTJiuceBottle
{
public:
    float get_cur_volume() const {
        std::lock_guard guard(mtx_);
        return cur_volume_;
    }
    
    void set_cur_volume(const float value) {
        std::lock_guard guard(mtx_);
        cur_volume_ = value;
    }
    
    float get_max_volume() const {
        std::lock_guard guard(mtx_);
        return max_volume_;
    }
    
    void set_max_volume(const float value) {
        std::lock_guard guard(mtx_);
        max_volume_ = value;
    }

private:
    float cur_volume_;
    float max_volume_;
    mutable std::mutex mtx_;
};

// thread 1
bottle.set_max_volume(100);
bottle.set_cur_volume(50);

// thread 2
bottle.set_cur_volume(25);
bottle.set_max_volume(25);
```

<details>
<summary>замечание</summary>
<p>

Поддержка инвариантов в классах с многопоточной поддержкой требуют особой тщательности при проектировании интерфейсов.

</p>
</details>

<br />

__Рекомендация__: минимизируйте число классов и объектов в программе, поддерживающих многопоточный доступ. Классы с поддержкой многопоточности сложнее проектировать и сложнее в использовании.

<br />

##### deadlock

https://en.wikipedia.org/wiki/Deadlock

__Вопрос__: что такое deadlock и как с ним бороться?

deadlock классический:
    
```c++
std::mutex m1; // мьютекс для ресурса 1
std::mutex m2; // мьютекс для ресурса 2

void worker_1()                  |  void worker_2()
{                                |  {
    std::lock_guard guard1(m1);  |      std::lock_guard guard2(m2);
    std::lock_guard guard2(m2);  |      std::lock_guard guard1(m1);
    ...;                         |      ...;
}                                |  }
```

__Вопрос__: как его починить?

__Ответ__:

```c++
std::mutex m1; // мьютекс для ресурса 1
std::mutex m2; // мьютекс для ресурса 2

void worker_1()                  |  void worker_2()
{                                |  {
    std::lock_guard guard1(m1);  |      std::lock_guard guard1(m1);
    std::lock_guard guard2(m2);  |      std::lock_guard guard2(m2);
    ...;                         |      ...;
}                                |  }
```

<br />

##### scoped_lock

Не всегда легко определить правильный порядок блокировки мьютексов.

Вспомним `MTQueue` и его `operator =`:

```c++
template<typename T>
class MTQueue
{
private:
    std::mutex mtx;
    std::queue<T> queue;

public:
    MTQueue& operator =(const MTQueue& rhs)
    {
        if (this != &rhs)
        {
            // что блокировать первым?
            // std::lock_guard guard_1(mtx);
            // std::lock_guard guard_2(res.mtx);
            queue = rhs.queue;
        }
        return *this;
    }
};

MTQueue<int> q1;
MTQueue<int> q2;

// thread 1:
q2 = q1;

// thread 2:
q1 = q2;
```

или, например:

```c++
bool operator == (const MTQueue& lhs, const MTQueue& rhs)
{
    // что блокировать первым?
    // std::lock_guard guard_1(lhs.mtx);
    // std::lock_guard guard_2(rhs.mtx);
    return lhs.queue == rhs.queue;
}

// thread 1:
q1 == q2


// thread 2:
q2 == q1
```

В таком коде нет правильного выбора последовательности блокировок

<br />

Для решения проблемы существует `std::scoped_lock`:

https://en.cppreference.com/w/cpp/thread/scoped_lock

```c++
bool operator == (const MTQueue& lhs, const MTQueue& rhs)
{
    std::scoped_lock guard(lhs.mtx, rhs.mtx);

    return lhs.queue == rhs.queue;
}
```

и:

```c++
template<typename T>
class MTQueue
{
private:
    std::mutex mtx;
    std::queue<T> queue;

public:
    MTQueue& operator =(const MTQueue& rhs)
    {
        if (this != &rhs)
        {
            std::scoped_lock guard(mtx, rhs.mtx);

            queue = rhs.queue;
        }
        return *this;
    }
};
```

<br />

Для варианта:

```c++
std::mutex m1;
std::mutex m2;

std::scoped_lock guard_1{m1, m2};  // thread 1
std::scoped_lock guard_2{m2, m1};  // thread 2
```

`std::scoped_lock` автоматически определяет порядок, таким образом, что `guard_1{m1, m2}` и `guard_2{m2, m1}` на разных потоках "эквивалентны" и не приведут к deadlock.

<br />

##### recursive_mutex

__Упражнение:__ Каким минимальным числом мьютексов и потоков можно добиться deadlock?


<details>
<summary>Ответ</summary>
<p>

```c++
int main()
{
    std::mutex m;
    m.lock();
    m.lock();
    return 0;
}
```

</p>
</details>

Зависание через дважды `lock()` на одном потоке - не самая приятная особенность.

Если, например, класс с многопоточным доступом простой, то все пути с блокировками `std::mutex` можно отследить глазами. Но есть случаи, когда:
* класс с многопоточным доступом спроектирован плохо, отследить все пути сложно
* класс в реализации методов управление отдаётся наружу, например:

```c++
class MTClass
{
private:
    std::mutex m;
    
public:
    void method()
    {
        std::lock_guard guard(m);
        
        global_funcion();  // <--- подозрительное место, не дойдёт ли в callstack
                           //      до вызова этого же метода снова? (нужно отслеживать)
    }
};
```

или так:

```c++
class TasksQueue
{
public:
    using Task = std::function<void(void)>;
    
    void push(Tash task)
    {
        std::lock_guard guard(mtx);
        
        tasks_queue.emplace_back(std::move(task));
    }
    
    void pop_and_run()
    {
        std::lock_guard guard(mtx);
        
        if (!tasks_queue.empty())
        {        
            const auto task = std::move(tasks_queue.back());
            tasks_queue.pop_back();
            task();  // <--- запускается неизвестная задача, которая может захотеть
                     //      поставить другую задачу в очередь через |TasksQueue::push|
        }        
    }
    
private:
    std::mutex mtx;
    std::queue<Task> tasks_queue;
};
```

Чтобы справляться с этими ситуациями нужен `std::recursive_mutex`

https://en.cppreference.com/w/cpp/thread/recursive_mutex

* хранит внутри счётчик, сколько раз был сделан `lock`
* каждый `lock` увеличивает счётчик на 1
* каждый `unlock` уменьшает счётчик на 1
* кол-во `unlock`-ов должно совпадать с кол-вом `lock`-ов
* максимальное значение счётчика - unspecified, при его превышении - exception

```c++
// отработает корректно:
int main()
{
    std::recursive_mutex mtx;
    mtx.lock();
    mtx.lock(); // ok
    mtx.unlock();
    mtx.unlock();
    return 0;
}
```

пример с `TasksQueue`:

```c++
class TasksQueue
{
public:
    using Task = std::function<void(void)>;

    void push(Tash task)
    {
        std::lock_guard guard(mtx);

        tasks_queue.emplace_back(std::move(task));
    }

    void pop_and_run()
    {
        std::lock_guard guard(mtx);

        if (!tasks_queue.empty())
        {        
            const auto task = std::move(tasks_queue.back());
            tasks_queue.pop_back();
            task();  // <--- ok
        }        
    }

private:
    std::recursive_mutex mtx;
    std::queue<Task> tasks_queue;
};
```

__Замечание__: Скорректированный пример с `TasksQueue` с обычным `std::mutex`:

```c++
class TasksQueue
{
public:
    using Task = std::function<void(void)>;

    void push(Tash task)
    {
        std::lock_guard guard(mtx);

        tasks_queue.emplace_back(std::move(task));
    }

    void pop_and_run()
    {
        Task task;
        
        {
            std::lock_guard guard(mtx);

            if (!tasks_queue.empty())
            {        
                task = std::move(tasks_queue.back());
                tasks_queue.pop_back();
            }
        }
        
        if (task)
            task();
    }

private:
    std::mutex mtx;
    std::queue<Task> tasks_queue;
};
```

<br />

##### shared_mutex, unique_lock, shared_lock

https://en.cppreference.com/w/cpp/thread/shared_mutex

https://en.cppreference.com/w/cpp/thread/unique_lock

https://en.cppreference.com/w/cpp/thread/shared_lock

__Вопрос__: кто-нибудь работал с RWLock? Что это такое и зачем оно нужно?

`std::shared_mutex` - похожая концепция, где read - это shared, а write - это exclusive
* позволяет захватывать себя в shared и exclusive режимах (чтения и записи)
* exclusive (писатель) единомоментно может быть только один, при этом не может быть shared (читателей)
* shared (читатель) единомоментно может быть несколько

Основные методы:

для exclusive (write) режима
* `lock`
* `try_lock`
* `unlock`

для shared (read) режима
* `lock_shared`
* `try_lock_shared`
* `unlock_shared`

RAII-захват:
* `std::unique_lock` захватывает в exclusive (write) режиме (можно `std::lock_guard` тоже)
* `std::shared_lock` захватывает в shared (read) режиме

```c++
class MTJiuceBottle
{
private:
    float cur_volume_;
    float max_volume_;
    mutable std::shared_mutex mtx_;

public:
    float get_cur_volume() const {
        std::shared_lock lock(mtx_, std::adopt_lock);
        return cur_volume_;
    }

    void set_cur_volume(const float value) {
        std::lock_guard guard(mtx_);
        cur_volume_ = value;
    }

    float get_max_volume() const {
        std::shared_lock lock(mtx_, std::adopt_lock);
        return max_volume_;
    }

    void set_max_volume(const float value) {
        std::lock_guard guard(mtx_);
        max_volume_ = value;
    }
};
```

<br />

##### condition_variable

https://en.cppreference.com/w/cpp/thread/condition_variable

https://en.cppreference.com/w/cpp/thread/condition_variable_any

https://en.wikipedia.org/wiki/Spurious_wakeup

`std::condition_variable` - примитив синхронизации, позволяющий одному потоку нотифицировать другой/другие, что какое-то событие произошло.

`std::condition_variable` работает только с `std::unique_lock<std::mutex>` (для эффективности). Если нужно что-то другое - можно использовать `std::condition_variable_any`

Принцип работы:
    
* есть условие `condition`, о смене которого нужно сообщить между потоками
* есть `std::mutex` для синхронизации, `condition` и его внутренности вычисляются только если мьютекс захвачен
* есть `std::condition_variable` для отсылки нотификации

методы:

* `notify_one` - отослать нотификацию одному потоку, что событие произошло / условие поменялось
* `notify_all` - отослать нотификацию всем потокам, что событие произошло / условие поменялось
* `wait(std::unique_lock<std::mutex>& lock)` - подождать, пока либо пока кто-нибудь нас не нотифицирует о событии либо ОКОНЧИТЬ ОЖИДАНИЕ ПРОСТО ТАК ПОТОМУ ЧТО ЗАХОТЕЛОСЬ, ДАЖЕ ЕСЛИ НИКТО НЕ ОТСЫЛАЛ НОТИФИКАЦИИ.
    * проблема в сложностях реализации честного `wait`.
* `wait( std::unique_lock<std::mutex>& lock, Predicate pred )` - эквивалент:

```c++
    while (!pred)
        wait(lock);
```

__Пример__:

```c++
std::mutex m;
std::condition_variable cv_ready;
std::condition_variable cv_compl;
std::string data;
bool is_inp_ready = false;
bool is_completed = false;
 
void worker_function()
{
    std::unique_lock lk(m);  // mutex locked
    cv_ready.wait(lk, []{ return is_inp_ready; });  // mutex unlocked, locked inside lambda
    // mutex locked

    process(data);
    is_completed = true;
    
    cv_compl.notify_one();
}
 
int main()
{
    std::thread thr(worker_function);
    
    data = prepare_input();
 
    {
        std::lock_guard guard(m);
        is_inp_ready = true;
    }
    cv_ready.notify_one();
 
    {
        std::unique_lock lk(m);
        cv_compl.wait(lk, []{ return is_completed; });
    }

    thr.join();
}
```

Код работает, но в нём проблема, в чём она?

Более правильный вариант:
    
```c++
std::mutex m;
std::condition_variable cv_ready;
std::condition_variable cv_compl;
std::string data;
bool is_inp_ready = false;
bool is_completed = false;

void worker_function()
{
    std::unique_lock lk(m);  // mutex locked
    cv_ready.wait(lk, []{ return is_inp_ready; });  // mutex unlocked, locked inside lambda
    // mutex locked

    process(data);
    is_completed = true;
    
    lk.unlock();  // !avoid redundant switches!
    cv_compl.notify_one();
}

int main()
{
    std::thread thr(worker_function);

    data = prepare_input();

    {
        std::lock_guard guard(m);
        is_inp_ready = true;
    }
    cv_ready.notify_one();

    {
        std::unique_lock lk(m);
        cv_compl.wait(lk, []{ return is_completed; });
    }

    thr.join();
}
```

<br />

##### thread_local

`thread_local` - модификатор перед переменной / константой, делающий её глобальной в рамках одного потока:

```c++
int func()
{
    thread_local unsigned i = 0;
    ++i;
    // в каждом потоке будет своя личная глобальная переменная i,
    // которая будет равна числу вызовов функции |func| на этом потоке
    
    ...;
}
```

__Пример__: кешированные данные с потокобезопасным доступом

```c++
std::string convert_to_string(const int number)
{
    // кеш памяти для конвертации, чтобы избежать частых переаллокаций
    thread_local std::string data_cache;
    
    // не нужно синхронизаций потоков для |data_cache|, т.к. у каждого
    // потока свой личный |data_cache|.
    convert_to_cached_location(data_cache, number);
    
    return data_cache;    
}
```

__Замечание__: это ученический пример, сконвертировать число в строку можно и другими более быстрыми способами.

Плюсы:
* возможность делать потокобезопасные глобальные кеши

Минусы:
* создание потока дороже, т.к. для каждого нового потока нужно вызывать конструкторы `thread_local` данных
    * хуже того, создание потока дороже неявно. Если вы слинковались с левой библиотекой, использующей `thread_local`-оптимизации для себя лично, то ВСЕ ваши потоки стали создаваться медленнее.
* старые операционки не поддерживают `thread_local` (не компиляторы, а операционки!). Например, Windows XP (поддерживает, начиная с Windows XP SP 3, емнип). В них `thread_local` молча становится `static` без синхронизаций, и программы неожиданно начинают ловить race condition.

<br />

##### Резюме

* `std::mutex` - один из вариантов избавления от race condition
* захватывать и освобождать `std::mutex` желательно через RAII: `std::lock_guard`
* объекты с многопоточным доступом требуют более тщательной проработки дизайна
* `std::scoped_lock` как вариант лечения deadlock
* `std::recursive_mutex` поможет, если нужно `std::mutex` захватить больше одного раза в одном потоке
* `std::shared_mutex / std::unique_lock / std::shared_lock` - для организации читателей и писателей
* `std::conditional_variable` - примитив синхронизации лоя нотификации других потоков о событиях (помните о spurios wakeups)
* `thread_local` - глобальные данные, видимые для одного потока (у каждого потока свои)

<br />

**Замечания:**
* добавить информацию об `std::osyncstream` (C++20) в районе вопроса потокобезопасности потоков